In [1]:
import numpy as np
import tensorflow as tf 
from collections import deque
import random
import pdb
from replay_memory_Q import ReplayMemory
from doubleq_networks import DoubleQ_Network
from test_game import test_env
from info_obj import info_holder
import os
from __future__ import print_function
from mountain_car_addon import addon
import time
try:
    import cPickle
except:
    import _pickle as cPickle

In [2]:
import gym
#game_name = 'LunarLander-v2'
game_name = 'CartPole-v0'
#game_name = 'MountainCar-v0'
game = gym.make(game_name)
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n
'''
game = test_env()
state_dim = 1
action_dim = 2
'''

[2017-01-03 17:30:19,386] Making new env: CartPole-v0


'\ngame = test_env()\nstate_dim = 1\naction_dim = 2\n'

In [3]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
if_load = 'n'
hid_lay = [64, 64]
save_dir = './Results/'+ game_name[:-3] + '/net/'
if 'LunarLander' in game_name:
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = 128
    LEARNING_RATE = 1e-3 #5e-5 lr/2
    lr_decay_rate = 1
    EPSILON = 1
    eps_decay_rate = 0.9
    C_target_update = 16 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(3e4)
    MAX_STEP = int(1e3)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    
elif 'CartPole' in game_name:
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = 128
    LEARNING_RATE = 1e-4 #5e-5 lr/2
    lr_decay_rate = 1
    EPSILON = 1
    eps_decay_rate = 0.955
    C_target_update = 8 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    
elif 'MountainCar' in game_name:
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = 128
    LEARNING_RATE = 1e-3 #5e-5 lr/2
    lr_decay_rate = 1
    EPSILON = 1
    eps_decay_rate = 0.9
    C_target_update = 100 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    adjustment_reward = -1

MIN_EPSILON = 0.1
#### load the network
logging_mode = 'w'
strings = ['Learning rate = ', 'lr decay = ', 'Epsilon = ', 'eps decay = ', 'Buffer size = ', 'Batch size = ',\
           'hidden layers: ', 'target update cycle: ']
if 'y' == if_load:
    if not(os.path.isdir(save_dir)):
        raise NameError('No network to upload')
    logging_mode = 'a'
    with open(save_dir + 'params.txt', 'r') as f:
        xa = f.readline()
        while xa:
            if strings[0] in xa:
                LEARNING_RATE = float(xa[len(strings[0]):])
            if strings[1] in xa:
                lr_decay_rate = float(xa[len(strings[1]):])
            elif strings[2] in xa:
                EPSILON = float(xa[len(strings[2]):])
            elif strings[3] in xa:
                eps_decay_rate = float(xa[len(strings[3]):])
            elif strings[4] in xa:
                BUFFER_SIZE = int(xa[len(strings[4]):])
            elif strings[5] in xa:
                MINIBATCH_SIZE = int(xa[len(strings[5]):])
            elif strings[6] in xa:
                hid_lay = [int(xb) for xb in xa[len(strings[6]):].split()]
            elif strings[7] in xa:
                C_target_update = int(xa[len(strings[7]):])
            xa = f.readline()
    with open(save_dir + 'rewards.txt', 'r') as f:
        num_lines = sum(1 for line in f)
        #xa = f.readline()
        #xa = xa.split(',')
        start_ep = num_lines - 1#int(xa[0]) + 1
    try:
        var = raw_input("From what episode?(Enter a number or last): ")
    except:
        var = input("From what episode?(Enter a number or last): ")
    if var != 'last': 
        var_int = int(var)
        while ((var_int%save_ep)!=0) or (var_int > start_ep):
            print('Episode not found!')
            try:
                var = raw_input("From what episode?(Enter a number or last): ")
            except:
                var = input("From what episode?(Enter a number or last): ")
        start_ep = var_int
    print('Loading the previous trained network and continue training from episode %d' %start_ep)
else:
    start_ep = 0
lr = LEARNING_RATE * (lr_decay_rate ** (start_ep/lr_decay_ep))# / MINIBATCH_SIZE
eps = EPSILON * (eps_decay_rate ** (start_ep/lr_decay_ep))
if eps < MIN_EPSILON:
    eps = MIN_EPSILON
MIN_EPSILON = 0.01
#### till here
if 'MountainCar' in game_name:
    MCaddon = addon(adjustment_reward)
buff = ReplayMemory(BUFFER_SIZE)
info = info_holder()
try:
    DQN
except:
    pass
else:
    DQN.reset_net()

graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.hid_layers = np.array(hid_lay)
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.entropy_beta = ENTROPY_BETA
    DQN = DoubleQ_Network(info)

In [4]:
#eps = EPSILON
#lr = LEARNING_RATE
buff.clear()
last_ep = 0
sol_found = 1
archived_reward = []
archived_loss = []
with tf.Session(graph=graph) as sess:
    DQN.set_up(sess)
    sess.run(tf.global_variables_initializer())
    
    if if_load == 'y':
        saver = tf.train.Saver()
        filename = save_dir + 'Episod_' + str(start_ep-1) + '.chk'
        saver.restore(sess, filename)
        buff.load_buf(save_dir)
        #with open(filename+'reward', 'rb') as p:  # Python 3: open(..., 'wb')
        #    [archived_reward, archived_loss] = cPickle.load(p)
    #pdb.set_trace()
    ep_reward = np.zeros(MAX_EPISODE)
    actual_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    batch_loss = np.zeros(MAX_EPISODE)
    buffer_time = np.zeros(MAX_EPISODE)
    train_time = np.zeros(MAX_EPISODE)
    #avg_var = 0
    
    for ep in range(MAX_EPISODE):
        s = game.reset()
        if 'MountainCar' in game_name:
            MCaddon.reset()
        #var = 0
        for st in range(MAX_STEP):
            a = DQN.take_action(s.reshape(-1,info.dim_s), eps)
            s2, r, terminal, report = game.step(a)
            if 'MountainCar' in game_name:
                actual_reward[ep] += (1.*r)
                r = r + MCaddon.reward_adjusment(a) * sol_found
            buff.add(np.reshape(s, (info.dim_s,)).astype(np.float32), np.array(a).astype(np.float32),\
                     np.array(r).astype(np.float32), np.reshape(s2, (info.dim_s,)).astype(np.float32),\
                     np.array(terminal).astype(np.float32))
            
            ep_reward[ep] += (1.*r)
            #var += np.sum(actor.give_policy(s.reshape(-1,info.dim_s))**2)
            s = s2
            if buff.size() >= MINIBATCH_SIZE:
                start = time.time()
                (s_batch, a_batch, r_batch, s2_batch, t_batch) = buff.sample_batch(MINIBATCH_SIZE)
                #start = time.time()
                s_batch = np.array(s_batch).reshape(-1, info.dim_s)
                s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
                a_batch = np.array(a_batch)#.reshape(-1, 1)
                r_batch = np.array(r_batch)#.reshape(-1, 1)
                t_batch = np.array(t_batch)#.reshape(-1, 1)
                buffer_time[ep] += (time.time() - start)
                start = time.time()
                Q_max = DQN.target_q(s2_batch)
                Q_max = Q_max[0]
                
                R_batch = r_batch + GAMMA * Q_max * (1-t_batch) # if terminal then the second part is zero

                if(len(a_batch.shape)>1):
                    pdb.set_trace()
                    print(a_batch.shape, buff.size(), ep, st)
                batch_loss[ep], _ = DQN.train(s_batch, a_batch, R_batch, lr)
                train_time[ep] += (time.time() - start)
            if terminal:
                buffer_time[ep] = buffer_time[ep]/st
                train_time[ep] = train_time[ep]/st
                break
        if terminal == False:
            buffer_time[ep] = buffer_time[ep]/MAX_STEP
            train_time[ep] = train_time[ep]/MAX_STEP
                
        if ((ep % C_target_update) == (C_target_update -1)):
            DQN.update_target()
                
        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):
            print('Ep: %d --- Ave R: %.1f  |  loss: %.1f  |  Suc: %.2f  |  crash: %.2f  |  Act R: %.1f  |  lr:%.0fe-4  |  eps:%.0fe-2  |  buffer_time: %f |  train_time: %f'\
                  %((start_ep+ep), np.mean(ep_reward[last_ep:ep+1]), np.mean(batch_loss[last_ep:ep+1]),\
                    np.mean(ep_success[last_ep:ep+1]), np.mean(ep_crash[last_ep:ep+1]), \
                    np.amax(ep_reward[last_ep:ep+1]), 1e4*lr, 1e2*eps, np.mean(buffer_time[last_ep:ep+1]), np.mean(train_time[last_ep:ep+1])))
            last_ep = ep
            #if(np.mean(actual_reward[last_ep:ep+1]) > -190):
            #    sol_found = 0
            
        if (ep == (save_ep-1)) and (if_load == 'n'):  ####
            os.mkdir(save_dir)  
            with open(save_dir + 'params.txt', 'w') as f:
                f.write('Learning rate = %f\n' %LEARNING_RATE)
                f.write('lr decay = %f\n' %lr_decay_rate)
                f.write('Epsilon = %f\n' %EPSILON)
                f.write('eps decay = %f\n' %eps_decay_rate)
                f.write('Buffer size = %d\n' %BUFFER_SIZE)
                f.write('Batch size = %d\n' %MINIBATCH_SIZE)
                f.write('hidden layers: %s\n' %str(info.hid_layers)[1:-1])
                f.write('target update cycle: %d\n' %C_target_update)
                
        if (ep == 0) and (if_load == 'n'):  ####
            if (os.path.isdir(save_dir)):
                raise NameError('Directory net exists')
                
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ...' %(start_ep+ep), end="")
            saver = tf.train.Saver()
            filename = save_dir + 'Episod_' + str(start_ep+ep) + '.chk'
            saver.save(sess, filename)
            #with open(save_dir+'reward', 'wb') as p:  # Python 3: open(..., 'wb')
            #    cPickle.dump([archived_reward+ep_reward[:ep+1], archived_loss+batch_loss[:ep+1]], p)
            if (ep == (save_ep-1)) and (if_load != 'y'):
                with open(save_dir + 'rewards.txt', 'w') as f:
                    f.write('x, , , , ,Learning rate = %f - lr decay = %f - Epsilon = %f - eps decay = %f - Buffer size = %d - Batch size = %d - hidden layers: %s - target update cycle: %d\n' \
                            %(LEARNING_RATE, lr_decay_rate, EPSILON, eps_decay_rate, BUFFER_SIZE, MINIBATCH_SIZE, str(info.hid_layers)[1:-1], C_target_update) )
            with open(save_dir + 'rewards.txt', 'a') as f: 
                for i in range(ep-save_ep+1, ep+1):
                    f.write('%f,      %f\n' %(ep_reward[i], batch_loss[i]))
            buff.save_buf(save_dir)
            print(' Done ******')
        
            
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = lr_decay_rate * lr
            eps = eps_decay_rate * eps
            if eps < MIN_EPSILON:
                eps = MIN_EPSILON
            
print('DONE') 
    

Ep: 0 --- Ave R: 43.0  |  loss: 0.0  |  Suc: 0.00  |  crash: 0.00  |  Act R: 43.0  |  lr:1e-4  |  eps:100e-2  |  buffer_time: 0.000000 |  train_time: 0.000000
Ep: 99 --- Ave R: 21.7  |  loss: 13.2  |  Suc: 0.00  |  crash: 0.00  |  Act R: 81.0  |  lr:1e-4  |  eps:100e-2  |  buffer_time: 0.000559 |  train_time: 0.001694
Ep: 199 --- Ave R: 24.6  |  loss: 4.3  |  Suc: 0.00  |  crash: 0.00  |  Act R: 76.0  |  lr:1e-4  |  eps:96e-2  |  buffer_time: 0.000600 |  train_time: 0.001708
Ep: 299 --- Ave R: 25.2  |  loss: 5.4  |  Suc: 0.00  |  crash: 0.00  |  Act R: 94.0  |  lr:1e-4  |  eps:91e-2  |  buffer_time: 0.000635 |  train_time: 0.001703
Ep: 399 --- Ave R: 31.4  |  loss: 5.5  |  Suc: 0.00  |  crash: 0.00  |  Act R: 108.0  |  lr:1e-4  |  eps:87e-2  |  buffer_time: 0.000627 |  train_time: 0.001704


KeyboardInterrupt: 

In [ ]:
for i in range(100, 110):
    print(i)